<a href="https://colab.research.google.com/github/RodrigoGuedesDP/Quadtrees-with-GIS/blob/main/Quadtrees_funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Estructura Básica del Quadtree
En JavaScript implementamos dos clases principales: `Boundary` (límite) y `Quadtree`. En Python sería así:

In [ ]:

class Boundary:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def contains(self, point):
        return (point.x >= self.x - self.width/2 and
                point.x < self.x + self.width/2 and
                point.y >= self.y - self.height/2 and
                point.y < self.y + self.height/2)

class Quadtree:
    def __init__(self, boundary, capacity=4, max_depth=10, depth=0):
        self.boundary = boundary
        self.capacity = capacity
        self.points = []
        self.divided = False
        self.max_depth = max_depth
        self.depth = depth

        # Referencias a los nodos hijos
        self.northeast = None
        self.northwest = None
        self.southeast = None
        self.southwest = None
```









### 2. Inserción de Puntos

La inserción verifica si un punto está dentro del límite del nodo actual. Si hay espacio y no se ha alcanzado la profundidad máxima, se agrega el punto. Si no hay espacio, se subdivide el nodo

In [ ]:

def insert(self, point):
    # Si el punto está fuera del límite, no se inserta
    if not self.boundary.contains(point):
        return False

    # Si hay espacio y no está dividido, insertar aquí
    if len(self.points) < self.capacity and not self.divided and self.depth < self.max_depth:
        self.points.append(point)
        return True

    # Si no está dividido y no alcanzó profundidad máxima, subdividir
    if not self.divided and self.depth < self.max_depth:
        self.subdivide()

    # Intentar insertar en los hijos
    if self.divided:
        return (self.northeast.insert(point) or
                self.northwest.insert(point) or
                self.southeast.insert(point) or
                self.southwest.insert(point))

    # Si alcanzó profundidad máxima, agregar aquí
    self.points.append(point)
    return True


### 3. Subdivisión de Nodos
Cuando un nodo alcanza su capacidad máxima, se divide en cuatro cuadrantes:

In [ ]:

def subdivide(self):
    x = self.boundary.x
    y = self.boundary.y
    w = self.boundary.width / 2
    h = self.boundary.height / 2
    next_depth = self.depth + 1

    # Crear límites para cada cuadrante
    ne = Boundary(x + w/2, y - h/2, w, h)
    nw = Boundary(x - w/2, y - h/2, w, h)
    se = Boundary(x + w/2, y + h/2, w, h)
    sw = Boundary(x - w/2, y + h/2, w, h)

    # Crear nodos hijos
    self.northeast = Quadtree(ne, self.capacity, self.max_depth, next_depth)
    self.northwest = Quadtree(nw, self.capacity, self.max_depth, next_depth)
    self.southeast = Quadtree(se, self.capacity, self.max_depth, next_depth)
    self.southwest = Quadtree(sw, self.capacity, self.max_depth, next_depth)

    # Redistribuir puntos existentes
    for point in self.points:
        self.northeast.insert(point) or \
        self.northwest.insert(point) or \
        self.southeast.insert(point) or \
        self.southwest.insert(point)

    self.points = []  # Vaciar puntos del nodo padre
    self.divided = True


### 4. Búsqueda de Puntos

La búsqueda encuentra todos los puntos dentro de un área específica:

In [ ]:

def query(self, range):
    found_points = []

    # Si el rango no intersecta con este nodo, retornar vacío
    if not self.boundary.intersects(range):
        return found_points

    # Verificar puntos en este nodo
    for point in self.points:
        if range.contains(point):
            found_points.append(point)

    # Si está dividido, buscar en los hijos
    if self.divided:
        found_points.extend(self.northeast.query(range))
        found_points.extend(self.northwest.query(range))
        found_points.extend(self.southeast.query(range))
        found_points.extend(self.southwest.query(range))

    return found_points


###5. Eliminación de Puntos
La eliminación busca y remueve un punto específico:

In [ ]:
def remove(self, point, compare_func=lambda p1, p2: p1.id == p2.id):
    # Si el punto está fuera del límite
    if not self.boundary.contains(point):
        return False

    # Buscar en los puntos de este nodo
    for i, p in enumerate(self.points):
        if compare_func(p, point):
            self.points.pop(i)
            return True

    # Si está dividido, buscar en los hijos
    if self.divided:
        return (self.northeast.remove(point, compare_func) or
                self.northwest.remove(point, compare_func) or
                self.southeast.remove(point, compare_func) or
                self.southwest.remove(point, compare_func))

    return False

###Explicación del Funcionamiento:
##Inserción:
Verifica si el punto está dentro del límite del nodo
Si hay espacio y no se alcanzó la profundidad máxima, inserta el punto
Si no hay espacio, subdivide el nodo y redistribuye los puntos
##Subdivisión:
Divide el espacio en cuatro cuadrantes iguales
Crea nuevos nodos para cada cuadrante
Redistribuye los puntos existentes a los nodos hijos apropiados
##Búsqueda:
Verifica si el área de búsqueda intersecta con el nodo actual
Recolecta puntos que están dentro del área de búsqueda
Recursivamente busca en los nodos hijos si existen
##Eliminación:
Busca el punto en el nodo actual
Si no se encuentra y el nodo está dividido, busca en los hijos
Elimina el punto cuando lo encuentra
La eficiencia de estas operaciones depende de la distribución de los puntos y la profundidad máxima permitida. En el mejor caso, las operaciones tienen una complejidad de O(log n), pero en el peor caso (cuando todos los puntos están muy cercanos) puede ser O(n).